<a href="https://colab.research.google.com/github/vektor8891/llm/blob/main/projects/12_bert_preparing/12_bert_preparing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Loading and Text Processing for BERT

## Tokenization and vocabulary building

### Tokenization

Components:

- `tokenizer`: tokenizes text
- `yield_tokens`: generator function that iterates through the data
- `word_dict`: defines special tokens used in text processing, such as padding `[PAD]`, class `[CLS]`, separator `[SEP]`, mask `[MASK]`, and unknown `[UNK]`
- `text_to_index`: converts text into a list of numerical indices based on the vocabulary
- `index_to_en`: translates sequence of indices back into readable English text.

Special tokens:

- **`CLS` (Classification Token)**: Start of Sentence (SOS) marker
- **`SEP` (Separator Token)**: End of Sentence (EOS) marker
- **`PAD` (Padding Token)**: added to sequences to ensure all inputs are of equal length
- **`MASK` (Masked Token)**: Utilized masked language modeling
- **`UNK` (Unknown Token)**: placeholder for unknown words

In [ ]:
print(4)